In [417]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import plotly.express as px
import datetime
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import zipfile

In [418]:
df = pd.read_csv("data/time_series_covid19_deaths_US.csv")
df.tail()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/11/22,4/12/22,4/13/22,4/14/22,4/15/22,4/16/22,4/17/22,4/18/22,4/19/22,4/20/22
3337,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,16,16,16,16,16,16,16,16,16,16
3338,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,39,39,39,39,39,39,39,39,39,39
3339,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3340,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,43,43,43,43,43,43,43,43,44,44
3341,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,...,18,18,18,18,18,18,18,18,18,18


In [419]:
df_time = (df
           .groupby("Province_State")
           .sum()
           .iloc[:, 12:]
           .T)
df_time.tail()

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
4/16/22,19502,1235,18,29823,11339,89670,12023,10813,2894,0,...,26109,87809,4736,623,111,20022,12619,6794,14390,1801
4/17/22,19502,1235,18,29823,11348,89670,12023,10813,2894,0,...,26109,87809,4736,623,111,20022,12619,6794,14390,1801
4/18/22,19510,1235,18,29823,11354,89697,12029,10813,2894,0,...,26109,87849,4736,623,111,20044,12626,6796,14391,1801
4/19/22,19513,1235,21,29823,11354,89797,12036,10813,2896,0,...,26110,87892,4736,626,111,20071,12626,6804,14395,1807
4/20/22,19524,1248,21,29852,11360,89838,12030,10825,2896,0,...,26109,87929,4736,626,111,20099,12650,6807,14399,1807


In [420]:
df_time_delta = (df_time
                 .diff()
                 .rolling(1)
                 .mean()
                 .fillna(0)
                 .iloc[1:, :])
df_time_delta.tail()

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
4/16/22,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4/17/22,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4/18/22,8.0,0.0,0.0,0.0,6.0,27.0,6.0,0.0,0.0,0.0,...,0.0,40.0,0.0,0.0,0.0,22.0,7.0,2.0,1.0,0.0
4/19/22,3.0,0.0,3.0,0.0,0.0,100.0,7.0,0.0,2.0,0.0,...,1.0,43.0,0.0,3.0,0.0,27.0,0.0,8.0,4.0,6.0
4/20/22,11.0,13.0,0.0,29.0,6.0,41.0,-6.0,12.0,0.0,0.0,...,-1.0,37.0,0.0,0.0,0.0,28.0,24.0,3.0,4.0,0.0


In [421]:
SE_states = ["Alabama", "Florida", "Georgia", "Arkansas", "Kentucky",
             "Louisiana", "Mississippi", "North Carolina", "South Carolina", "Tennessee"]
NE_states = ["Maine", "New York", "New Jersey", "Delaware", "Vermont",
             "Massachusetts", "Rhode Island", "Connecticut", "New Hampshire", "Pennsylvania"]

In [422]:
average_southeast = df_time_delta[SE_states].sum(axis=1)
average_northeast = df_time_delta[NE_states].sum(axis=1)

In [423]:
south_v_north = (pd
                 .concat([average_northeast, average_southeast],
                         axis=1)
                 .rename(columns={0: "Northeast",
                                  1: "Southeast"})
                )

In [424]:
roll = 14
fig = px.line(south_v_north.rolling(roll).mean(),
              title=f"Southeast vs. Northeast, Daily Deaths Count, {roll} Day Rolling Average"
              )
fig.show()

In [425]:
base_url = "https://www2.census.gov/programs-surveys/popest"
population = pd.read_excel(base_url + "/tables/2020-2021/state/totals/NST-EST2021-POP.xlsx", skiprows=2)

In [426]:
population = population.iloc[6:57, :]

In [427]:
population.loc[:, "State"] = (population
                              .loc[:, "Geographic Area"]
                              .str
                              .strip(".")
                             )
population_isolated = population[["State", "April 1, 2020 Estimates Base"]]
population_isolated = population_isolated.set_index("State")

In [428]:
per_capita = df_time_delta[population_isolated.T.columns].div(population_isolated.T.iloc[0, :], axis=1) * 100

In [429]:
average_southeast_pc = per_capita[SE_states].sum(axis=1)
average_northeast_pc = per_capita[NE_states].sum(axis=1)
south_v_north_pc = (pd
                    .concat([average_northeast_pc, average_southeast_pc], axis=1)
                    .rename(columns={0: "Northeast", 1: "Southeast"})
                   )

In [430]:
roll = 14
fig = px.line(south_v_north_pc.rolling(roll).mean(),
              title=f"Southeast vs. Northeast, Deaths per 100 people, {roll} Day Rolling Average"); 
fig.show()

In [431]:
fig = px.line(average_southeast_pc.rolling(30).corr(average_northeast_pc),
              title="Southeast vs. Northeast, Correlation Coefficient over past 14 Days"); 
fig.show()

### Retrieving Census Pulse Data

In [432]:
df["st_code"] = df["FIPS"].astype(str).apply(lambda x: "0" + x if len(x) < 7 else x).str[:2]
state_codes = df[["st_code", "Province_State"]].drop_duplicates(subset="Province_State")

In [433]:
zf = zipfile.ZipFile('data/HPS_Week34_PUF_CSV.zip') 
pulse_1 = pd.read_csv(zf.open('pulse2021_puf_34.csv'))

In [434]:
SE_codes = state_codes[state_codes["Province_State"].isin(SE_states)]["st_code"].astype(int)
NE_codes = state_codes[state_codes["Province_State"].isin(NE_states)]["st_code"].astype(int)

In [435]:
pulse_1_SE = pulse_1[pulse_1["EST_ST"].isin(SE_codes)]
pulse_1_NE = pulse_1[pulse_1["EST_ST"].isin(NE_codes)]

In [436]:
pulse_1_SE = pulse_1_SE.replace({-88: None, -99: None})

In [437]:
pulse_1_NE = pulse_1_NE.replace({-88: None, -99: None})

Note, including Income will lower the sample size too much.

$\text{Anxious or Depressed}_i = \beta_0 + \beta_1\text{Health_Insurance}_i + \beta_2\text{Race}_i + \beta_3\text{Education}_i + \beta_4\text{Marital_Status}_i + \beta_5\text{Sexual_Orientation}_i + \beta_6\text{Age}_i + \beta_7\text{Vaccination_Status}_i + \beta_8\text{Plan_to_Vaccinate}_i + \beta_8\text{Work_Loss}_i + \beta_9\text{CTC}_i + \beta_{10}\text{Difficulty_with_Expenses}_i + \beta_{11}\text{Work_on_Site}_i + \beta_{12}\text{Work_from_Home}_i + \beta_{13}\text{Food_Insecurity}_i \beta_{14}\text{Home_Owner}_i + \beta_{15}\text{Fear_of_Eviction}_i + \beta_{16}\vec{\text{State}}$

In [444]:
vars_of_interest = ["ANXIOUS",
                    "DOWN",
                    "HLTHINS1",
                    "HLTHINS2",
                    "HLTHINS3",
                    "HLTHINS4",
                    "HLTHINS5",
                    "HLTHINS6",
                    "HLTHINS7",
                    "HLTHINS8",
                    "RRACE",
                    "EEDUC",
                    "MS",
                    "SEXUAL_ORIENTATION",
                    "RHISPANIC",
                    "TBIRTH_YEAR",
                    "RECVDVACC",
                    "GETVACRV",
                    "WRKLOSSRV",
                    "CTC_YN",
                    "EXPNS_DIF",
                    "ACTIVITY1",
                    "ACTIVITY1",
                    "CURFOODSUF",
                    "TENURE",
                    "EVICT",]
pulse_1_SE_vars = pulse_1_SE.dropna(subset=vars_of_interest)[vars_of_interest]
pulse_1_NE_vars = pulse_1_NE.dropna(subset=vars_of_interest)[vars_of_interest]

In [446]:
for i in range(len(vars_of_interest)):
    print(pulse_1_NE.dropna(subset=vars_of_interest[:i]).shape)

(10488, 202)
(9237, 202)
(9212, 202)
(8686, 202)
(8075, 202)
(8013, 202)
(7871, 202)
(7814, 202)
(7785, 202)
(7751, 202)
(7476, 202)
(7476, 202)
(7476, 202)
(7454, 202)
(7375, 202)
(7375, 202)
(7375, 202)
(7365, 202)
(582, 202)
(581, 202)
(580, 202)
(580, 202)
(577, 202)
(577, 202)
(576, 202)
(534, 202)
(58, 202)
